In [240]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime

In [241]:
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)

In [242]:
from support_functions import portfolio, data_loader

In [243]:
importlib.reload(data_loader)

<module 'support_functions.data_loader' from '/Users/yifanli/Github/fidelity-portfolio-tracker/code/support_functions/data_loader.py'>

In [244]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'

transactions = data_loader.load_transaction(data_folder_path, transaction_file_pattern)
position = data_loader.load_position(data_folder_path, position_file_pattern)

# Attribute 1: get investment distribution

In [256]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [257]:
current_portfolio.individual_position["Description"].str.contains("BILLS")

0     False
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25      NaN
Name: Description, dtype: object

In [262]:
current_portfolio.get_other_investment()

13351.729999999981

In [263]:
current_portfolio.get_investment_distribution()

{'Class': ['stock', 'bill', 'other', 'total'],
 'Amount': [240727.72000000003, 345920.55, 13351.729999999981, 600000.0],
 'Percent': [0.4012128666666667, 0.57653425, 0.022252883333333303, 1.0]}

In [131]:
current_portfolio.show_investment_distribution()

   Class     Amount  Percent
0  stock  240727.72   40.12%
1   bill  316079.98   52.68%
2  other   43192.30    7.20%
3  total  600000.00  100.00%


# Attribute 2: get overall IRR

In [26]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [27]:
current_portfolio.get_overall_irr()

0.12285282721063749

# Attribute 3: get IRR for each stock

In [264]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [265]:
current_portfolio.show_stock_irr()

ValueError: f(a) and f(b) must have different signs

In [267]:
current_portfolio.add_total_current_value_to_individual_position()
current_portfolio.set_merged_individual_position_transaction()
current_portfolio.add_time_diff_in_merged_individual_position_transaction()
unique_symbols = current_portfolio.merged_individual_position_transaction["Symbol"].unique()
stock_list = [
    element
    for element in unique_symbols
    if "FZFXX" not in element and not element[0].isdigit()
]
stock_list

Total current value has been added


['Transfer',
 'AAPL',
 'SBUX',
 'JPM',
 'AXP',
 'GOOGL',
 'AMZN',
 'NKE',
 'TSLA',
 'FXAIX',
 'MCD',
 'FSKAX',
 'FSPSX',
 'COKE',
 'MSFT',
 'BRKB',
 'Pending Activity']

In [224]:
temp = current_portfolio.individual_transactions
temp[temp['Symbol']=='Transfer']['Amount ($)'].sum()

600000.0

In [225]:
current_portfolio.get_total_current_value()

0.0

In [226]:
current_portfolio.individual_position

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
0,Z23390746,Individual,FZFXX**,HELD IN MONEY MARKET,NaN,NaN,NaN,53946.02,NaN,NaN,NaN,NaN,8.41%,NaN,NaN,Cash
1,Z23390746,Individual,912796ZV4,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 12...,30000,$99.46,+$0.011,29838.00,+$3.30,+0.01%,-$2.57,-0.01%,4.65%,29840.57,--,Cash
2,Z23390746,Individual,912797HP5,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 11...,50000,$99.802,+$0.013,49901.00,+$6.50,+0.01%,+$537.33,+1.08%,7.78%,49363.67,--,Cash
3,Z23390746,Individual,912797KA4,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 02...,20000,$98.79,+$0.004,19758.00,+$0.80,0.00%,+$240.30,+1.23%,3.08%,19517.70,--,Cash
4,Z23390746,Individual,912797LE5,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 11...,50000,$99.901,+$0.012,49950.50,+$6.00,+0.01%,+$589.40,+1.19%,7.79%,49361.10,--,Cash
5,Z23390746,Individual,912797LP0,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 12...,20000,$99.639,+$0.013,19927.80,+$2.60,+0.01%,+$35.83,+0.18%,3.11%,19891.97,--,Cash
6,Z23390746,Individual,912797LR6,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,30000,$99.38,+$0.012,29814.00,+$3.60,+0.01%,+$155.25,+0.52%,4.65%,29658.75,--,Cash
7,Z23390746,Individual,912797LX3,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,50000,$99.294,+$0.011,49647.00,+$5.50,+0.01%,+$222.07,+0.44%,7.74%,49424.93,--,Cash
8,Z23390746,Individual,912797LY1,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,50000,$99.212,+$0.013,49606.00,+$6.50,+0.01%,+$176.65,+0.35%,7.73%,49429.35,--,Cash
9,Z23390746,Individual,912797LZ8,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,50000,$99.045,+$0.011,49522.50,+$5.50,+0.01%,+$89.99,+0.18%,7.72%,49432.51,--,Cash


In [166]:
import glob
position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
position_files = glob.glob(position_file_path_pattern)

In [ ]:
latest_file = None
latest_date = None

for file_path in position_files:
    # Extract date part from the filename
    file_name = os.path.basename(file_path)
    date_str = file_name.split("_")[-1].replace(".csv", "")
    file_date = datetime.strptime(date_str, "%b-%d-%Y")

    # Update the latest file if this date is more recent
    if latest_date is None or file_date > latest_date:
        latest_date = file_date
        latest_file = file_path

Load the most recent file as a DataFrame if found
if latest_file:
    df = pd.read_csv(latest_file)
    print(f"Loaded file: {latest_file}")
    print(df.head())
else:
    print("No matching files found.")

In [168]:
latest_file

'./data/Portfolio_Positions_Nov-12-2024.csv'